In [1]:
import sys
import time
import numpy as np
sys.path.append("../../Util")
from ProgressBar import ProgressBar
from Timing import Timing

In [2]:
with open("../../_Data/prices.txt", "r") as file:
    data = np.array([line.strip().split(",") for line in file], dtype=np.float32)
data[:3]

array([[   2104.,  399900.],
       [   1600.,  329900.],
       [   2400.,  369000.]], dtype=float32)

In [3]:
length = len(data)
n_train, n_cv = int(0.7*length), int(0.15*length)
idx = np.random.permutation(length)
train_idx, cv_idx = idx[:n_train], idx[n_train:n_train+n_cv]
test_idx = idx[n_train+n_cv:]
train, cv, test = data[train_idx], data[cv_idx], data[test_idx]
print(len(train), len(cv), len(test), length)

32 7 8 47


In [4]:
letters = np.array(list("qwertyuiopasdfghjklzxcvbnm"))
data = letters[np.random.randint(0, 26, 10000)]
features = set(data)
feat_dic = {c: i for i, c in enumerate(sorted(features))}

def transform(data, feat_dic):
    return [feat_dic[c] for c in data]

print(data[:3], transform(data, feat_dic)[:3])

['j' 'c' 'c'] [9, 2, 2]


In [5]:
data = ["a", "c", "b"]
print(data, transform(data, feat_dic))

['a', 'c', 'b'] [0, 2, 1]


In [6]:
def task(cost=0.5, epoch=3, name="", sub_task=None):
    def sub():
        bar = ProgressBar(max_value=epoch, name=name)
        for _ in range(epoch):
            time.sleep(cost)
            if sub_task is not None:
                sub_task()
            bar.update()
    return sub

task(name="Task1", sub_task=task(
    name="Task2", sub_task=task(
        name="Task3")))()


## #    Task1     # Progress bar initialized  ##
## #    Task2     # Progress bar initialized  ##
## #    Task3     # (3 : 0 -> 3) Task Finished. Time Cost:   0 h   0 min  1.512 s; Average:   0 h   0 min  0.504 s  ##
## #    Task2     # [----------                    ] : 1 / 3 ##  Time Cost:   0 h   0 min  2.016 s; Average:   0 h   0 min  2.016 s 
## #    Task3     # (3 : 0 -> 3) Task Finished. Time Cost:   0 h   0 min  1.512 s; Average:   0 h   0 min  0.504 s  ##
## #    Task2     # [--------------------          ] : 2 / 3 ##  Time Cost:   0 h   0 min  4.032 s; Average:   0 h   0 min  2.016 s 
## #    Task3     # (3 : 0 -> 3) Task Finished. Time Cost:   0 h   0 min   1.51 s; Average:   0 h   0 min 0.5034 s  ##
## #    Task2     # (3 : 0 -> 3) Task Finished. Time Cost:   0 h   0 min  6.046 s; Average:   0 h   0 min  2.015 s  ##
## #    Task1     # [----------                    ] : 1 / 3 ##  Time Cost:   0 h   0 min  6.552 s; Average:   0 h   0 min  6.552 s 
## #    Task2     # Progre

In [7]:
class Test:
    timing = Timing()

    def __init__(self, rate):
        self.rate = rate

    @timing.timeit()
    def test(self, cost=0.1, epoch=3):
        for _ in range(epoch):
            self._test(cost * self.rate)

    @timing.timeit(prefix="[Core] ")
    def _test(self, cost):
        time.sleep(cost)

class Test1(Test):
    def __init__(self):
        Test.__init__(self, 1)

class Test2(Test):
    def __init__(self):
        Test.__init__(self, 2)

class Test3(Test):
    def __init__(self):
        Test.__init__(self, 3)

test1 = Test1()
test2 = Test2()
test3 = Test3()
test1.test()
test2.test()
test3.test()
test1.timing.show_timing_log()


Timing log
--------------------------------------------------------------------------------------------------------------
             Test1                   [Core]                        _test :     0.3105171 s (Call Time:      3)
             Test1                 [Method]                         test :     0.3106821 s (Call Time:      1)
             Test2                   [Core]                        _test :     0.6009197 s (Call Time:      3)
             Test2                 [Method]                         test :     0.6014001 s (Call Time:      1)
             Test3                   [Core]                        _test :     0.9126101 s (Call Time:      3)
             Test3                 [Method]                         test :      0.912776 s (Call Time:      1)
--------------------------------------------------------------------------------------------------------------
